In [12]:
import numpy
from networkx import Graph, gnp_random_graph, convert_node_labels_to_integers, set_node_attributes, connected_components, compose, isolates, spring_layout, draw_networkx, draw_networkx_nodes,draw_networkx_edges,adjacency_matrix
from epydemic import ERNetwork, NetworkGenerator, BondPercolation 
from epyc import LabNotebook, HDF5LabNotebook, Lab, ParallelLab
from math import ceil
import random
# import matplotlib
# %matplotlib inline
# %config InlineBackend.figure_format = 'png'
# matplotlib.rcParams['figure.dpi'] = 300
# import matplotlib.pyplot as plt
# import seaborn
# matplotlib.style.use('seaborn')
# seaborn.set_context("notebook", font_scale=0.25)
#plotting.BACKEND = 'matplotlib'

### The Core-periphery Network Class

In [13]:
class CorePeripheryNetwork(NetworkGenerator):
    # Network parameters
    N_1 = 'coreperiphery.core.N' #: Experimental parameter for the number of nodes in the core network
    PHI_1 = 'coreperiphery.core.phi' #: Experimental parameter for the density in the core network
    N_2 = 'coreperiphery.periphery.N' #: Experimental parameter for the number of nodes in the periphery network
    PHI_2 = 'coreperiphery.periphery.phi' #: Experimental parameter for the density in the periphery network
    P_CONNECTIVITY='coreperiphery.connectivity' #: Experimental parameter for the connectivity between core and periphery
    
    # Node attributes
    ORIGIN = 'origin'
    def __init__(self, params=None, limit=None ):
        super(CorePeripheryNetwork, self).__init__(params, limit) 
        
        
    def topology(self):
        return 'ER-core-periphery'
    
    def _generate(self, params):
        rng = numpy.random.default_rng()
        pConn = params[self.P_CONNECTIVITY]
        
        # generate the core network
        N_core = params[self.N_1]
        phi_core = params[self.PHI_1]
        g_core = gnp_random_graph(N_core, phi_core)
        
        # generate the periphery network
        N_per = params[self.N_2]
        phi_per = params[self.PHI_2]
        g_per = gnp_random_graph(N_per, phi_per)
        
        # relabel the nodes into a single sequence
        g_core = convert_node_labels_to_integers(g_core, first_label=0)
        g_per = convert_node_labels_to_integers(g_per, first_label=N_core)
       
        # form both networks into a a single network, currently disconnected
        g = compose(g_core, g_per)

        # label all nodes with their origins
        for n in g_core.nodes():
            g.nodes[n][self.ORIGIN] = 0     # core nodes have origin == 0
        for m in g_per.nodes():
            g.nodes[m][self.ORIGIN] = 1     # periphery nodes have origin == 1
            
        # join the periphery to the core using the input pConn
        for n in g_core.nodes():
            for m in g_per.nodes():
                if rng.random() < pConn:
                    g.add_edge(n, m)             # edges added to composed network
            
        # restrict to the LCC
        lcc = g.subgraph(max(connected_components(g), key=len)).copy()
        lcc = convert_node_labels_to_integers(lcc, first_label=0)
        
        return lcc

In [14]:
from epydemic import SIR, ProcessSequence, Monitor, ERNetwork, StochasticDynamics
from epyc import Experiment
import numpy
class MonitoredSIR(SIR):

    def __init__(self):
        super().__init__()

    def build(self, params):
        '''Build the process, adding additional loci
        to be monitored.

        :param params: the parameters'''
        super().build(params)

        # add loci for the other compartments
        self.trackNodesInCompartment(SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(SIR.REMOVED)

### Run experiment using parallelLab
1. We run each 3 times and take average 
2. We use ParallelLab to parallel run experiments in order to save time
3. We save the results and experiments information into a JSON file

In [26]:
# select five values from 0.1 to 0.99 for P_connectivity
pConnectivity = numpy.linspace(0.10, 0.99 , num=5, endpoint=True)

In [27]:
#Run experiments
# Parameters set here

from epyc import ParallelLab, JSONLabNotebook,RepeatedExperiment
pnb = JSONLabNotebook('core-periphery-experiments-14-0545.json') # save results in a json file
plab = ParallelLab(pnb, cores= -1)  # run experiments parallelly 

plab[CorePeripheryNetwork.N_1] = 5000 # set core nodes number
plab[CorePeripheryNetwork.PHI_1] = 0.06 # set density of core cluster
plab[CorePeripheryNetwork.N_2] = 15000 # set periphery nodes number
plab[CorePeripheryNetwork.PHI_2] = 0.04 # set density of periphery cluster
plab[CorePeripheryNetwork.P_CONNECTIVITY] = 0.545 # set values for P_connectivity

# set disease parameter P_infected, P_infect, and P_remove
plab[SIR.P_INFECTED] = 0.0005
plab[SIR.P_INFECT] = 0.001
plab[SIR.P_REMOVE] = 0.002

# capture results every 5 timesteps
# build a compund process from the disease and the monitor
plab[Monitor.DELTA] = 5
p = ProcessSequence([MonitoredSIR(), Monitor()])

# use stochastic simulation to run experiments with maximum time 1000 set
# run experiments three times to address randomness
g = CorePeripheryNetwork()
e = StochasticDynamics(p, g)
e.process().setMaximumTime(1000)
n=3
plab.runExperiment(RepeatedExperiment(e, n))
df = plab.dataframe()

### Changing p_infect
Use five different Pconnectivity. Change P_infect and see the number of removal.

In [28]:
pConnectivity = numpy.linspace(0.10, 0.99 , num=5, endpoint=True)

from epyc import ParallelLab, JSONLabNotebook,RepeatedExperiment
pnb = JSONLabNotebook('core-periphery-experiments-20.json')
plab = ParallelLab(pnb, cores=-1)

plab[CorePeripheryNetwork.N_1] = 500 # set core nodes number
plab[CorePeripheryNetwork.PHI_1] = 0.06 # set density of core cluster
plab[CorePeripheryNetwork.N_2] = 1500 # set periphery nodes number
plab[CorePeripheryNetwork.PHI_2] = 0.04 set density of periphery cluster
plab[CorePeripheryNetwork.P_CONNECTIVITY] = pConnectivity # set values for P_connectivity


# set disease parameter P_infected, P_infect, and P_remove
plab[SIR.P_INFECTED] = 0.0005
plab[SIR.P_INFECT] = numpy.linspace(0.00001, 0.0009,
                                            num=50)
plab[SIR.P_REMOVE] = 0.002

# capture results every 5 timesteps
# build a compund process from the disease and the monitor
plab[Monitor.DELTA] = 5
p = ProcessSequence([MonitoredSIR(), Monitor()])

# use stochastic simulation to run experiments with maximum time 1000 set
# run experiments three times to address randomness
g = CorePeripheryNetwork()
e = StochasticDynamics(p, g)
e.process().setMaximumTime(1000)
n=10
plab.runExperiment(RepeatedExperiment(e, n))
df = plab.dataframe()

No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
